# This Notebook is an Updated version of my previous kernel https://www.kaggle.com/kushal1506/moa-pytorch-feature-engineering-0-01846

# If U find my work helpful and consider forking it, please do Upvote :)

In [1]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from pickle import load,dump

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.preprocessing import QuantileTransformer

In [4]:
os.listdir('../input/lish-moa')

['lish-moa.zip',
 'sample_submission.csv',
 'test_features.csv',
 'train_features.csv',
 'train_targets_nonscored.csv',
 'train_targets_scored.csv',
 'train_features_dummy.csv',
 'test_features_dummy.csv',
 'train_drug.csv',
 'main_predictors.json',
 '.ipynb_checkpoints']

In [5]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
df = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [6]:
train_features2=train_features.copy()
test_features2=test_features.copy()

In [7]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [8]:
for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [9]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
seed_everything(42)

In [10]:
n_comp = 600  #<--Update
pca_g = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
gpca= (pca_g.fit(data[GENES]))
train2= (gpca.transform(train_features[GENES]))
test2 = (gpca.transform(test_features[GENES]))

train_gpca = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test_gpca = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train_gpca), axis=1)
test_features = pd.concat((test_features, test_gpca), axis=1)

dump(gpca, open('gpca.pkl', 'wb'))

In [11]:
#CELLS
n_comp = 50  #<--Update

pca_c = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
cpca= (pca_c.fit(data[CELLS]))
train2= (cpca.transform(train_features[CELLS]))
test2 = (cpca.transform(test_features[CELLS]))

train_cpca = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test_cpca = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train_cpca), axis=1)
test_features = pd.concat((test_features, test_cpca), axis=1)

dump(cpca, open('cpca.pkl', 'wb'))

In [12]:
from sklearn.feature_selection import VarianceThreshold

c_n = [f for f in list(train_features.columns) if f not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
mask = (train_features[c_n].var() >= 0.85).values
tmp = train_features[c_n].loc[:, mask]
train_features = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)
tmp = test_features[c_n].loc[:, mask]
test_features = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)

In [13]:
from sklearn.cluster import KMeans
def fe_cluster_genes(train, test, n_clusters_g = 22, SEED = 42):
    
    features_g = GENES
    #features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans_genes = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        dump(kmeans_genes, open('kmeans_genes.pkl', 'wb'))
        train[f'clusters_{kind}'] = kmeans_genes.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_genes.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
   # train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_genes(train_features2,test_features2)

In [14]:
def fe_cluster_cells(train, test, n_clusters_c = 4, SEED = 42):
    
    #features_g = GENES
    features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'c', n_clusters = n_clusters_c):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans_cells = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        dump(kmeans_cells, open('kmeans_cells.pkl', 'wb'))
        train[f'clusters_{kind}'] = kmeans_cells.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_cells.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
   # train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_cells(train_features2,test_features2)

In [15]:
train_pca=pd.concat((train_gpca,train_cpca),axis=1)
test_pca=pd.concat((test_gpca,test_cpca),axis=1)

In [16]:
def fe_cluster_pca(train, test,n_clusters=5,SEED = 42):
        data=pd.concat([train,test],axis=0)
        kmeans_pca = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        dump(kmeans_pca, open('kmeans_pca.pkl', 'wb'))
        train[f'clusters_pca'] = kmeans_pca.predict(train.values)
        test[f'clusters_pca'] = kmeans_pca.predict(test.values)
        train = pd.get_dummies(train, columns = [f'clusters_pca'])
        test = pd.get_dummies(test, columns = [f'clusters_pca'])
        return train, test
train_cluster_pca ,test_cluster_pca = fe_cluster_pca(train_pca,test_pca)

In [17]:
train_cluster_pca = train_cluster_pca.iloc[:,650:]
test_cluster_pca = test_cluster_pca.iloc[:,650:]

In [18]:
train_features_cluster=train_features2.iloc[:,876:]
test_features_cluster=test_features2.iloc[:,876:]

In [19]:

gsquarecols=['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203','g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22','g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

In [20]:
def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1)
        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
        df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        df['g_skew'] = df[features_g].skew(axis = 1)
        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
        df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        df['gc_std'] = df[features_g + features_c].std(axis = 1)
        df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
        df['c52_c42'] = df['c-52'] * df['c-42']
        df['c13_c73'] = df['c-13'] * df['c-73']
        df['c26_c13'] = df['c-23'] * df['c-13']
        df['c33_c6'] = df['c-33'] * df['c-6']
        df['c11_c55'] = df['c-11'] * df['c-55']
        df['c38_c63'] = df['c-38'] * df['c-63']
        df['c38_c94'] = df['c-38'] * df['c-94']
        df['c13_c94'] = df['c-13'] * df['c-94']
        df['c4_c52'] = df['c-4'] * df['c-52']
        df['c4_c42'] = df['c-4'] * df['c-42']
        df['c13_c38'] = df['c-13'] * df['c-38']
        df['c55_c2'] = df['c-55'] * df['c-2']
        df['c55_c4'] = df['c-55'] * df['c-4']
        df['c4_c13'] = df['c-4'] * df['c-13']
        df['c82_c42'] = df['c-82'] * df['c-42']
        df['c66_c42'] = df['c-66'] * df['c-42']
        df['c6_c38'] = df['c-6'] * df['c-38']
        df['c2_c13'] = df['c-2'] * df['c-13']
        df['c62_c42'] = df['c-62'] * df['c-42']
        df['c90_c55'] = df['c-90'] * df['c-55']
        
        
        for feature in features_c:
             df[f'{feature}_squared'] = df[feature] ** 2     
                
        for feature in gsquarecols:
            df[f'{feature}_squared'] = df[feature] ** 2        
        
    return train, test

train_features2,test_features2=fe_stats(train_features2,test_features2)

In [21]:
train_features_stats=train_features2.iloc[:,902:]
test_features_stats=test_features2.iloc[:,902:]

In [22]:
train_features = pd.concat((train_features, train_features_cluster,train_cluster_pca,train_features_stats), axis=1)
test_features = pd.concat((test_features, test_features_cluster,test_cluster_pca,test_features_stats), axis=1)

In [23]:
train = train_features.merge(train_targets_nonscored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_nonscored.columns]

In [24]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [25]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [26]:
train = pd.get_dummies(train, columns=['cp_time','cp_dose'])
test_ = pd.get_dummies(test, columns=['cp_time','cp_dose'])

In [27]:
feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id']]

In [28]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [29]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

In [30]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [31]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x), 1e-3)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [32]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 7
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [33]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    mskf = MultilabelStratifiedKFold(n_splits=7,random_state=seed)
    for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
         train.loc[v_idx, 'kfold'] = int(f)
    train['kfold'] = train['kfold'].astype(int)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"SEED: {seed} ,FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_nonscored.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    model.load_state_dict(torch.load(f"SEED{seed}_FOLD{fold}_nonscored.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


In [34]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [35]:
# Averaging on multiple SEEDS

SEED = [0,1,2,3,4,5,6]  #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof


SEED: 0, FOLD: 0, EPOCH: 0, train_loss: 0.5904558038187994
SEED: 0 ,FOLD: 0, EPOCH: 0, valid_loss: 0.07420812776455513
SEED: 0, FOLD: 0, EPOCH: 1, train_loss: 0.01617622677538846
SEED: 0 ,FOLD: 0, EPOCH: 1, valid_loss: 0.005086700897663832
SEED: 0, FOLD: 0, EPOCH: 2, train_loss: 0.008451678884542874
SEED: 0 ,FOLD: 0, EPOCH: 2, valid_loss: 0.004860620623311171
SEED: 0, FOLD: 0, EPOCH: 3, train_loss: 0.008478350824097524
SEED: 0 ,FOLD: 0, EPOCH: 3, valid_loss: 0.004607470312084143
SEED: 0, FOLD: 0, EPOCH: 4, train_loss: 0.008247776598845786
SEED: 0 ,FOLD: 0, EPOCH: 4, valid_loss: 0.004823135426984384
SEED: 0, FOLD: 0, EPOCH: 5, train_loss: 0.008359443138328355
SEED: 0 ,FOLD: 0, EPOCH: 5, valid_loss: 0.0047693040508490344
SEED: 0, FOLD: 0, EPOCH: 6, train_loss: 0.008195578875775272
SEED: 0 ,FOLD: 0, EPOCH: 6, valid_loss: 0.004567584255710244
SEED: 0, FOLD: 0, EPOCH: 7, train_loss: 0.00817216297838132
SEED: 0 ,FOLD: 0, EPOCH: 7, valid_loss: 0.0045470910039372165
SEED: 0, FOLD: 0, EPOCH: 8,

SEED: 0, FOLD: 4, EPOCH: 5, train_loss: 0.00816450544280579
SEED: 0 ,FOLD: 4, EPOCH: 5, valid_loss: 0.004842119577985544
SEED: 0, FOLD: 4, EPOCH: 6, train_loss: 0.00815237803082611
SEED: 0 ,FOLD: 4, EPOCH: 6, valid_loss: 0.004876140254334762
SEED: 0, FOLD: 4, EPOCH: 7, train_loss: 0.00813583327406967
SEED: 0 ,FOLD: 4, EPOCH: 7, valid_loss: 0.004819209687411785
SEED: 0, FOLD: 4, EPOCH: 8, train_loss: 0.00813225457897863
SEED: 0 ,FOLD: 4, EPOCH: 8, valid_loss: 0.004658960092526216
SEED: 0, FOLD: 4, EPOCH: 9, train_loss: 0.008140169864369405
SEED: 0 ,FOLD: 4, EPOCH: 9, valid_loss: 0.004849371679413777
SEED: 0, FOLD: 4, EPOCH: 10, train_loss: 0.008130235452758702
SEED: 0 ,FOLD: 4, EPOCH: 10, valid_loss: 0.004757759519494497
SEED: 0, FOLD: 4, EPOCH: 11, train_loss: 0.00814237487124833
SEED: 0 ,FOLD: 4, EPOCH: 11, valid_loss: 0.004795052684270418
SEED: 0, FOLD: 4, EPOCH: 12, train_loss: 0.008134346568604579
SEED: 0 ,FOLD: 4, EPOCH: 12, valid_loss: 0.004850281330828483
SEED: 0, FOLD: 4, EPOCH

SEED: 1, FOLD: 1, EPOCH: 10, train_loss: 0.008129815854182517
SEED: 1 ,FOLD: 1, EPOCH: 10, valid_loss: 0.004683618397953419
SEED: 1, FOLD: 1, EPOCH: 11, train_loss: 0.008138140990726045
SEED: 1 ,FOLD: 1, EPOCH: 11, valid_loss: 0.004792651949593654
SEED: 1, FOLD: 1, EPOCH: 12, train_loss: 0.008120597195434006
SEED: 1 ,FOLD: 1, EPOCH: 12, valid_loss: 0.004816119881490102
SEED: 1, FOLD: 1, EPOCH: 13, train_loss: 0.008116548125808304
SEED: 1 ,FOLD: 1, EPOCH: 13, valid_loss: 0.0047953258841656726
SEED: 1, FOLD: 1, EPOCH: 14, train_loss: 0.00812589514975411
SEED: 1 ,FOLD: 1, EPOCH: 14, valid_loss: 0.004769701331567306
SEED: 1, FOLD: 1, EPOCH: 15, train_loss: 0.008121651194586948
SEED: 1 ,FOLD: 1, EPOCH: 15, valid_loss: 0.004717934590119582
SEED: 1, FOLD: 2, EPOCH: 0, train_loss: 0.5885558985173702
SEED: 1 ,FOLD: 2, EPOCH: 0, valid_loss: 0.07413001702382015
SEED: 1, FOLD: 2, EPOCH: 1, train_loss: 0.015830115289301484
SEED: 1 ,FOLD: 2, EPOCH: 1, valid_loss: 0.005347323138266802
SEED: 1, FOLD: 

SEED: 1, FOLD: 5, EPOCH: 15, train_loss: 0.00812181384881606
SEED: 1 ,FOLD: 5, EPOCH: 15, valid_loss: 0.004784429839883859
SEED: 1, FOLD: 5, EPOCH: 16, train_loss: 0.008100779834077568
SEED: 1 ,FOLD: 5, EPOCH: 16, valid_loss: 0.004776625535809076
SEED: 1, FOLD: 5, EPOCH: 17, train_loss: 0.008089483791106456
SEED: 1 ,FOLD: 5, EPOCH: 17, valid_loss: 0.004695617629644962
SEED: 1, FOLD: 6, EPOCH: 0, train_loss: 0.5886718759665618
SEED: 1 ,FOLD: 6, EPOCH: 0, valid_loss: 0.08996332952609429
SEED: 1, FOLD: 6, EPOCH: 1, train_loss: 0.015777207948770876
SEED: 1 ,FOLD: 6, EPOCH: 1, valid_loss: 0.005357192578510596
SEED: 1, FOLD: 6, EPOCH: 2, train_loss: 0.008481917474922296
SEED: 1 ,FOLD: 6, EPOCH: 2, valid_loss: 0.004882686216240892
SEED: 1, FOLD: 6, EPOCH: 3, train_loss: 0.008366398462975348
SEED: 1 ,FOLD: 6, EPOCH: 3, valid_loss: 0.004866712475911929
SEED: 1, FOLD: 6, EPOCH: 4, train_loss: 0.008244523828899538
SEED: 1 ,FOLD: 6, EPOCH: 4, valid_loss: 0.004798901912111502
SEED: 1, FOLD: 6, EPOC

SEED: 2, FOLD: 3, EPOCH: 2, train_loss: 0.008502176471960705
SEED: 2 ,FOLD: 3, EPOCH: 2, valid_loss: 0.0055242541532676956
SEED: 2, FOLD: 3, EPOCH: 3, train_loss: 0.00960401857171107
SEED: 2 ,FOLD: 3, EPOCH: 3, valid_loss: 0.004633207214423097
SEED: 2, FOLD: 3, EPOCH: 4, train_loss: 0.008411466767004615
SEED: 2 ,FOLD: 3, EPOCH: 4, valid_loss: 0.004640069378253359
SEED: 2, FOLD: 3, EPOCH: 5, train_loss: 0.008253890951793338
SEED: 2 ,FOLD: 3, EPOCH: 5, valid_loss: 0.004780857573048427
SEED: 2, FOLD: 3, EPOCH: 6, train_loss: 0.008200374929033019
SEED: 2 ,FOLD: 3, EPOCH: 6, valid_loss: 0.0047720443421544936
SEED: 2, FOLD: 3, EPOCH: 7, train_loss: 0.008184262633172644
SEED: 2 ,FOLD: 3, EPOCH: 7, valid_loss: 0.004786834443131318
SEED: 2, FOLD: 3, EPOCH: 8, train_loss: 0.008160497118542725
SEED: 2 ,FOLD: 3, EPOCH: 8, valid_loss: 0.004696331846599395
SEED: 2, FOLD: 3, EPOCH: 9, train_loss: 0.008156805078977265
SEED: 2 ,FOLD: 3, EPOCH: 9, valid_loss: 0.004683255791091002
SEED: 2, FOLD: 3, EPOCH

...

SEED: 4, FOLD: 2, EPOCH: 8, train_loss: 0.00812249691729006
SEED: 4 ,FOLD: 2, EPOCH: 8, valid_loss: 0.004715307866438077
SEED: 4, FOLD: 2, EPOCH: 9, train_loss: 0.00814116739265218
SEED: 4 ,FOLD: 2, EPOCH: 9, valid_loss: 0.004667642329318019
SEED: 4, FOLD: 2, EPOCH: 10, train_loss: 0.008136711857357138
SEED: 4 ,FOLD: 2, EPOCH: 10, valid_loss: 0.004749804412802825
SEED: 4, FOLD: 2, EPOCH: 11, train_loss: 0.008106480673509272
SEED: 4 ,FOLD: 2, EPOCH: 11, valid_loss: 0.004679119393516045
SEED: 4, FOLD: 2, EPOCH: 12, train_loss: 0.008100896129837713
SEED: 4 ,FOLD: 2, EPOCH: 12, valid_loss: 0.004811348930860941
SEED: 4, FOLD: 2, EPOCH: 13, train_loss: 0.008093338812122474
SEED: 4 ,FOLD: 2, EPOCH: 13, valid_loss: 0.004705982067837165
SEED: 4, FOLD: 2, EPOCH: 14, train_loss: 0.008096495039156965
SEED: 4 ,FOLD: 2, EPOCH: 14, valid_loss: 0.004716857551382138
SEED: 4, FOLD: 2, EPOCH: 15, train_loss: 0.008068795731240834
SEED: 4 ,FOLD: 2, EPOCH: 15, valid_loss: 0.004714421318987241
SEED: 4, FOLD:

SEED: 4, FOLD: 6, EPOCH: 12, train_loss: 0.008141098223072855
SEED: 4 ,FOLD: 6, EPOCH: 12, valid_loss: 0.004783063876227691
SEED: 4, FOLD: 6, EPOCH: 13, train_loss: 0.008129557714218626
SEED: 4 ,FOLD: 6, EPOCH: 13, valid_loss: 0.004655313409435062
SEED: 4, FOLD: 6, EPOCH: 14, train_loss: 0.00812666689523974
SEED: 4 ,FOLD: 6, EPOCH: 14, valid_loss: 0.004672668659343169
SEED: 4, FOLD: 6, EPOCH: 15, train_loss: 0.008116627766473873
SEED: 4 ,FOLD: 6, EPOCH: 15, valid_loss: 0.004683449416636274
SEED: 5, FOLD: 0, EPOCH: 0, train_loss: 0.590108752351355
SEED: 5 ,FOLD: 0, EPOCH: 0, valid_loss: 0.07954944326327397
SEED: 5, FOLD: 0, EPOCH: 1, train_loss: 0.015901921493177477
SEED: 5 ,FOLD: 0, EPOCH: 1, valid_loss: 0.005299071888797558
SEED: 5, FOLD: 0, EPOCH: 2, train_loss: 0.00848446345283977
SEED: 5 ,FOLD: 0, EPOCH: 2, valid_loss: 0.005051566216234977
SEED: 5, FOLD: 0, EPOCH: 3, train_loss: 0.008506605584124053
SEED: 5 ,FOLD: 0, EPOCH: 3, valid_loss: 0.00491568145270531
SEED: 5, FOLD: 0, EPOCH

SEED: 5, FOLD: 3, EPOCH: 13, train_loss: 0.008112003724720027
SEED: 5 ,FOLD: 3, EPOCH: 13, valid_loss: 0.00475227481757219
SEED: 5, FOLD: 4, EPOCH: 0, train_loss: 0.5890284305488741
SEED: 5 ,FOLD: 4, EPOCH: 0, valid_loss: 0.08624497801065445
SEED: 5, FOLD: 4, EPOCH: 1, train_loss: 0.015853538895277557
SEED: 5 ,FOLD: 4, EPOCH: 1, valid_loss: 0.0051660316709715584
SEED: 5, FOLD: 4, EPOCH: 2, train_loss: 0.008475937672915894
SEED: 5 ,FOLD: 4, EPOCH: 2, valid_loss: 0.004897074702267463
SEED: 5, FOLD: 4, EPOCH: 3, train_loss: 0.008323430096277513
SEED: 5 ,FOLD: 4, EPOCH: 3, valid_loss: 0.021911580330477312
SEED: 5, FOLD: 4, EPOCH: 4, train_loss: 0.008318853237338969
SEED: 5 ,FOLD: 4, EPOCH: 4, valid_loss: 0.005435212372014156
SEED: 5, FOLD: 4, EPOCH: 5, train_loss: 0.008186083488367699
SEED: 5 ,FOLD: 4, EPOCH: 5, valid_loss: 0.0048103292042819355
SEED: 5, FOLD: 4, EPOCH: 6, train_loss: 0.008155860466177802
SEED: 5 ,FOLD: 4, EPOCH: 6, valid_loss: 0.004810531695301716
SEED: 5, FOLD: 4, EPOCH:

SEED: 6, FOLD: 1, EPOCH: 4, train_loss: 0.008252946521482757
SEED: 6 ,FOLD: 1, EPOCH: 4, valid_loss: 0.004694781660173948
SEED: 6, FOLD: 1, EPOCH: 5, train_loss: 0.008182199873231552
SEED: 6 ,FOLD: 1, EPOCH: 5, valid_loss: 0.0052050154727812
SEED: 6, FOLD: 1, EPOCH: 6, train_loss: 0.008159238473839453
SEED: 6 ,FOLD: 1, EPOCH: 6, valid_loss: 0.004738672815549832
SEED: 6, FOLD: 1, EPOCH: 7, train_loss: 0.008151262149063719
SEED: 6 ,FOLD: 1, EPOCH: 7, valid_loss: 0.004580571853484099
SEED: 6, FOLD: 1, EPOCH: 8, train_loss: 0.008131031462973034
SEED: 6 ,FOLD: 1, EPOCH: 8, valid_loss: 0.004671221204961722
SEED: 6, FOLD: 1, EPOCH: 9, train_loss: 0.008131258951097325
SEED: 6 ,FOLD: 1, EPOCH: 9, valid_loss: 0.0047283373916378385
SEED: 6, FOLD: 1, EPOCH: 10, train_loss: 0.008127875909259593
SEED: 6 ,FOLD: 1, EPOCH: 10, valid_loss: 0.004630480606395464
SEED: 6, FOLD: 1, EPOCH: 11, train_loss: 0.008141167543677462
SEED: 6 ,FOLD: 1, EPOCH: 11, valid_loss: 0.004753724851000767
SEED: 6, FOLD: 1, EPO

SEED: 6, FOLD: 5, EPOCH: 12, train_loss: 0.008122780882577235
SEED: 6 ,FOLD: 5, EPOCH: 12, valid_loss: 0.004683979500371676
SEED: 6, FOLD: 5, EPOCH: 13, train_loss: 0.008098258995576887
SEED: 6 ,FOLD: 5, EPOCH: 13, valid_loss: 0.004710670274037581
SEED: 6, FOLD: 5, EPOCH: 14, train_loss: 0.008083943296475587
SEED: 6 ,FOLD: 5, EPOCH: 14, valid_loss: 0.00471742211196285
SEED: 6, FOLD: 5, EPOCH: 15, train_loss: 0.00807935669640633
SEED: 6 ,FOLD: 5, EPOCH: 15, valid_loss: 0.004702119264178551
SEED: 6, FOLD: 5, EPOCH: 16, train_loss: 0.008083788331950436
SEED: 6 ,FOLD: 5, EPOCH: 16, valid_loss: 0.004707254469394684
SEED: 6, FOLD: 5, EPOCH: 17, train_loss: 0.00804384401999414
SEED: 6 ,FOLD: 5, EPOCH: 17, valid_loss: 0.004659973335667298
SEED: 6, FOLD: 5, EPOCH: 18, train_loss: 0.008016865694190602
SEED: 6 ,FOLD: 5, EPOCH: 18, valid_loss: 0.004676724282594828
SEED: 6, FOLD: 6, EPOCH: 0, train_loss: 0.5887856607300204
SEED: 6 ,FOLD: 6, EPOCH: 0, valid_loss: 0.0866628443965545
SEED: 6, FOLD: 6,

In [36]:
for col in target_cols:
    test_[col] = 0
    
test_[target_cols] = predictions

In [37]:
train = train.merge(train_targets_scored, on='sig_id')
target = train[train_targets_scored.columns]
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [38]:
feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id','kfold']]

In [39]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 7
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [40]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    mskf = MultilabelStratifiedKFold(n_splits=7,random_state=seed)
    for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
         train.loc[v_idx, 'kfold'] = int(f)
    train['kfold'] = train['kfold'].astype(int)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"SEED: {seed} ,FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_scored.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"SEED{seed}_FOLD{fold}_scored.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


In [41]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [42]:
# Averaging on multiple SEEDS

SEED = [0,1,2,3,4,5,6]  #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
# test[target_cols] = predictions

SEED: 0, FOLD: 0, EPOCH: 0, train_loss: 0.5874745306131002
SEED: 0 ,FOLD: 0, EPOCH: 0, valid_loss: 0.09026707594211285
SEED: 0, FOLD: 0, EPOCH: 1, train_loss: 0.02962773381354841
SEED: 0 ,FOLD: 0, EPOCH: 1, valid_loss: 0.020357302175118372
SEED: 0, FOLD: 0, EPOCH: 2, train_loss: 0.021836849055378825
SEED: 0 ,FOLD: 0, EPOCH: 2, valid_loss: 0.02149292969932923
SEED: 0, FOLD: 0, EPOCH: 3, train_loss: 0.021831931621843093
SEED: 0 ,FOLD: 0, EPOCH: 3, valid_loss: 0.017837081104516983
SEED: 0, FOLD: 0, EPOCH: 4, train_loss: 0.020540433331719926
SEED: 0 ,FOLD: 0, EPOCH: 4, valid_loss: 0.017711039202717636
SEED: 0, FOLD: 0, EPOCH: 5, train_loss: 0.02022745183392151
SEED: 0 ,FOLD: 0, EPOCH: 5, valid_loss: 0.017539642607936494
SEED: 0, FOLD: 0, EPOCH: 6, train_loss: 0.020108500898287102
SEED: 0 ,FOLD: 0, EPOCH: 6, valid_loss: 0.017265335871623114
SEED: 0, FOLD: 0, EPOCH: 7, train_loss: 0.020000001390439434
SEED: 0 ,FOLD: 0, EPOCH: 7, valid_loss: 0.01730056952398557
SEED: 0, FOLD: 0, EPOCH: 8, tra

SEED: 0, FOLD: 2, EPOCH: 15, train_loss: 0.01929389327965878
SEED: 0 ,FOLD: 2, EPOCH: 15, valid_loss: 0.01653318780546005
SEED: 0, FOLD: 2, EPOCH: 16, train_loss: 0.019140252239398053
SEED: 0 ,FOLD: 2, EPOCH: 16, valid_loss: 0.016381659974845555
SEED: 0, FOLD: 2, EPOCH: 17, train_loss: 0.018917652526618662
SEED: 0 ,FOLD: 2, EPOCH: 17, valid_loss: 0.0163642202432339
SEED: 0, FOLD: 2, EPOCH: 18, train_loss: 0.01863364769598922
SEED: 0 ,FOLD: 2, EPOCH: 18, valid_loss: 0.01618037337007431
SEED: 0, FOLD: 2, EPOCH: 19, train_loss: 0.018311426018339558
SEED: 0 ,FOLD: 2, EPOCH: 19, valid_loss: 0.01611223816871643
SEED: 0, FOLD: 2, EPOCH: 20, train_loss: 0.01791205141391303
SEED: 0 ,FOLD: 2, EPOCH: 20, valid_loss: 0.016055414739709634
SEED: 0, FOLD: 2, EPOCH: 21, train_loss: 0.017479128607020184
SEED: 0 ,FOLD: 2, EPOCH: 21, valid_loss: 0.016041627941796414
SEED: 0, FOLD: 2, EPOCH: 22, train_loss: 0.017124901047429524
SEED: 0 ,FOLD: 2, EPOCH: 22, valid_loss: 0.015952457172366288
SEED: 0, FOLD: 2

SEED: 0, FOLD: 5, EPOCH: 4, train_loss: 0.02057464637265012
SEED: 0 ,FOLD: 5, EPOCH: 4, valid_loss: 0.019361456139729574
SEED: 0, FOLD: 5, EPOCH: 5, train_loss: 0.020161710427822294
SEED: 0 ,FOLD: 5, EPOCH: 5, valid_loss: 0.017104041046248034
SEED: 0, FOLD: 5, EPOCH: 6, train_loss: 0.020083977110885286
SEED: 0 ,FOLD: 5, EPOCH: 6, valid_loss: 0.01690872056552997
SEED: 0, FOLD: 5, EPOCH: 7, train_loss: 0.019998568689098228
SEED: 0 ,FOLD: 5, EPOCH: 7, valid_loss: 0.016713350724715453
SEED: 0, FOLD: 5, EPOCH: 8, train_loss: 0.019996070705756947
SEED: 0 ,FOLD: 5, EPOCH: 8, valid_loss: 0.016894793782669764
SEED: 0, FOLD: 5, EPOCH: 9, train_loss: 0.01993325231848536
SEED: 0 ,FOLD: 5, EPOCH: 9, valid_loss: 0.01676392089575529
SEED: 0, FOLD: 5, EPOCH: 10, train_loss: 0.0198798201273422
SEED: 0 ,FOLD: 5, EPOCH: 10, valid_loss: 0.01691589905665471
SEED: 0, FOLD: 5, EPOCH: 11, train_loss: 0.01979886886437197
SEED: 0 ,FOLD: 5, EPOCH: 11, valid_loss: 0.01650337051027096
SEED: 0, FOLD: 5, EPOCH: 12, 

SEED: 1 ,FOLD: 0, EPOCH: 19, valid_loss: 0.016352426117429368
SEED: 1, FOLD: 0, EPOCH: 20, train_loss: 0.0178103904944618
SEED: 1 ,FOLD: 0, EPOCH: 20, valid_loss: 0.016148427716241434
SEED: 1, FOLD: 0, EPOCH: 21, train_loss: 0.017379634409538797
SEED: 1 ,FOLD: 0, EPOCH: 21, valid_loss: 0.016166392140663587
SEED: 1, FOLD: 0, EPOCH: 22, train_loss: 0.016990852919784753
SEED: 1 ,FOLD: 0, EPOCH: 22, valid_loss: 0.016094819356042605
SEED: 1, FOLD: 0, EPOCH: 23, train_loss: 0.016605494229274022
SEED: 1 ,FOLD: 0, EPOCH: 23, valid_loss: 0.016124126573021594
SEED: 1, FOLD: 0, EPOCH: 24, train_loss: 0.016328165722960555
SEED: 1 ,FOLD: 0, EPOCH: 24, valid_loss: 0.016110453181541882
SEED: 1, FOLD: 0, EPOCH: 25, train_loss: 0.01622094414071054
SEED: 1 ,FOLD: 0, EPOCH: 25, valid_loss: 0.016126501875428054
SEED: 1, FOLD: 1, EPOCH: 0, train_loss: 0.5887733867039552
SEED: 1 ,FOLD: 1, EPOCH: 0, valid_loss: 0.08381833479954647
SEED: 1, FOLD: 1, EPOCH: 1, train_loss: 0.029596281791659625
SEED: 1 ,FOLD: 1,

SEED: 1 ,FOLD: 3, EPOCH: 8, valid_loss: 0.017398000050049562
SEED: 1, FOLD: 3, EPOCH: 9, train_loss: 0.0198552236464378
SEED: 1 ,FOLD: 3, EPOCH: 9, valid_loss: 0.017313161291755162
SEED: 1, FOLD: 3, EPOCH: 10, train_loss: 0.019812119671621838
SEED: 1 ,FOLD: 3, EPOCH: 10, valid_loss: 0.017353404599886674
SEED: 1, FOLD: 3, EPOCH: 11, train_loss: 0.01978311186807381
SEED: 1 ,FOLD: 3, EPOCH: 11, valid_loss: 0.01732786501256319
SEED: 1, FOLD: 3, EPOCH: 12, train_loss: 0.019732860168693838
SEED: 1 ,FOLD: 3, EPOCH: 12, valid_loss: 0.01717411196575715
SEED: 1, FOLD: 3, EPOCH: 13, train_loss: 0.01959842609593997
SEED: 1 ,FOLD: 3, EPOCH: 13, valid_loss: 0.017043899816389266
SEED: 1, FOLD: 3, EPOCH: 14, train_loss: 0.019465656321797822
SEED: 1 ,FOLD: 3, EPOCH: 14, valid_loss: 0.017090127875025455
SEED: 1, FOLD: 3, EPOCH: 15, train_loss: 0.01928311140855422
SEED: 1 ,FOLD: 3, EPOCH: 15, valid_loss: 0.01681245190019791
SEED: 1, FOLD: 3, EPOCH: 16, train_loss: 0.019096596493712953
SEED: 1 ,FOLD: 3, E

...

SEED: 5, FOLD: 3, EPOCH: 21, train_loss: 0.017522695147105166
SEED: 5 ,FOLD: 3, EPOCH: 21, valid_loss: 0.015988406008825853
SEED: 5, FOLD: 3, EPOCH: 22, train_loss: 0.017120809470479552
SEED: 5 ,FOLD: 3, EPOCH: 22, valid_loss: 0.01597343349399475
SEED: 5, FOLD: 3, EPOCH: 23, train_loss: 0.016715751532968635
SEED: 5 ,FOLD: 3, EPOCH: 23, valid_loss: 0.01594116839651878
SEED: 5, FOLD: 3, EPOCH: 24, train_loss: 0.01644065190811415
SEED: 5 ,FOLD: 3, EPOCH: 24, valid_loss: 0.015970666892826557
SEED: 5, FOLD: 3, EPOCH: 25, train_loss: 0.016306157639199816
SEED: 5 ,FOLD: 3, EPOCH: 25, valid_loss: 0.015962225528290637
SEED: 5, FOLD: 4, EPOCH: 0, train_loss: 0.5875586721341353
SEED: 5 ,FOLD: 4, EPOCH: 0, valid_loss: 0.08507648970072086
SEED: 5, FOLD: 4, EPOCH: 1, train_loss: 0.02946030318334296
SEED: 5 ,FOLD: 4, EPOCH: 1, valid_loss: 0.01988325096093691
SEED: 5, FOLD: 4, EPOCH: 2, train_loss: 0.02250568483125519
SEED: 5 ,FOLD: 4, EPOCH: 2, valid_loss: 0.018640016563809834
SEED: 5, FOLD: 4, EPOCH

SEED: 5, FOLD: 6, EPOCH: 11, train_loss: 0.019761458190309035
SEED: 5 ,FOLD: 6, EPOCH: 11, valid_loss: 0.01760320752285994
SEED: 5, FOLD: 6, EPOCH: 12, train_loss: 0.019700207747519016
SEED: 5 ,FOLD: 6, EPOCH: 12, valid_loss: 0.01687811902509286
SEED: 5, FOLD: 6, EPOCH: 13, train_loss: 0.01953842319749497
SEED: 5 ,FOLD: 6, EPOCH: 13, valid_loss: 0.0168568747691237
SEED: 5, FOLD: 6, EPOCH: 14, train_loss: 0.019491104358756863
SEED: 5 ,FOLD: 6, EPOCH: 14, valid_loss: 0.01673226388028035
SEED: 5, FOLD: 6, EPOCH: 15, train_loss: 0.019347932718291477
SEED: 5 ,FOLD: 6, EPOCH: 15, valid_loss: 0.01663294105002513
SEED: 5, FOLD: 6, EPOCH: 16, train_loss: 0.019110821992964357
SEED: 5 ,FOLD: 6, EPOCH: 16, valid_loss: 0.016673254780471325
SEED: 5, FOLD: 6, EPOCH: 17, train_loss: 0.01894194519499669
SEED: 5 ,FOLD: 6, EPOCH: 17, valid_loss: 0.016586237372114107
SEED: 5, FOLD: 6, EPOCH: 18, train_loss: 0.018563089942609943
SEED: 5 ,FOLD: 6, EPOCH: 18, valid_loss: 0.01631006161467387
SEED: 5, FOLD: 6,

SEED: 6, FOLD: 2, EPOCH: 0, train_loss: 0.5886393593171159
SEED: 6 ,FOLD: 2, EPOCH: 0, valid_loss: 0.09404443089778607
SEED: 6, FOLD: 2, EPOCH: 1, train_loss: 0.029979535177148676
SEED: 6 ,FOLD: 2, EPOCH: 1, valid_loss: 0.01993798163647835
SEED: 6, FOLD: 2, EPOCH: 2, train_loss: 0.021803646172220643
SEED: 6 ,FOLD: 2, EPOCH: 2, valid_loss: 0.018758472628318347
SEED: 6, FOLD: 2, EPOCH: 3, train_loss: 0.021151924128266605
SEED: 6 ,FOLD: 2, EPOCH: 3, valid_loss: 0.01774847636429163
SEED: 6, FOLD: 2, EPOCH: 4, train_loss: 0.020498323581508687
SEED: 6 ,FOLD: 2, EPOCH: 4, valid_loss: 0.017327106915987454
SEED: 6, FOLD: 2, EPOCH: 5, train_loss: 0.02031073799809894
SEED: 6 ,FOLD: 2, EPOCH: 5, valid_loss: 0.017740181718881313
SEED: 6, FOLD: 2, EPOCH: 6, train_loss: 0.020092783923688774
SEED: 6 ,FOLD: 2, EPOCH: 6, valid_loss: 0.01758751617028163
SEED: 6, FOLD: 2, EPOCH: 7, train_loss: 0.02000199187849019
SEED: 6 ,FOLD: 2, EPOCH: 7, valid_loss: 0.0169444359265841
SEED: 6, FOLD: 2, EPOCH: 8, train_

SEED: 6 ,FOLD: 4, EPOCH: 18, valid_loss: 0.016494091027058087
SEED: 6, FOLD: 4, EPOCH: 19, train_loss: 0.018301559838692884
SEED: 6 ,FOLD: 4, EPOCH: 19, valid_loss: 0.016395683113772135
SEED: 6, FOLD: 4, EPOCH: 20, train_loss: 0.017971199962335663
SEED: 6 ,FOLD: 4, EPOCH: 20, valid_loss: 0.01631458863042868
SEED: 6, FOLD: 4, EPOCH: 21, train_loss: 0.01753636187798268
SEED: 6 ,FOLD: 4, EPOCH: 21, valid_loss: 0.016223304331875764
SEED: 6, FOLD: 4, EPOCH: 22, train_loss: 0.017152983405803506
SEED: 6 ,FOLD: 4, EPOCH: 22, valid_loss: 0.016230562773461524
SEED: 6, FOLD: 4, EPOCH: 23, train_loss: 0.016780697086171525
SEED: 6 ,FOLD: 4, EPOCH: 23, valid_loss: 0.016198294667097237
SEED: 6, FOLD: 4, EPOCH: 24, train_loss: 0.016482273506856448
SEED: 6 ,FOLD: 4, EPOCH: 24, valid_loss: 0.016214168487259976
SEED: 6, FOLD: 4, EPOCH: 25, train_loss: 0.01642646040213672
SEED: 6 ,FOLD: 4, EPOCH: 25, valid_loss: 0.01619551287820706
SEED: 6, FOLD: 5, EPOCH: 0, train_loss: 0.58713971340173
SEED: 6 ,FOLD: 5,

KeyError: "None of [Index(['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor',\n       'acat_inhibitor', 'acetylcholine_receptor_agonist',\n       'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor',\n       'adenosine_receptor_agonist', 'adenosine_receptor_antagonist',\n       'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist',\n       ...\n       'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist',\n       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',\n       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',\n       'vitamin_d_receptor_agonist', 'wnt_inhibitor'],\n      dtype='object', length=206)] are in the [columns]"

In [43]:
for col in target_cols:
    test[col] = 0
test[target_cols] = predictions

In [44]:

valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
print('oof shape:',valid_results.shape)
y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0

for i in range(len(target_cols)):
    score += log_loss(y_true[:, i], y_pred[:, i])

print("CV log_loss: ", score / y_pred.shape[1])

oof shape: (23814, 207)
CV log_loss:  0.01453437105181852


In [45]:
valid_results.to_csv('oof_model5.csv', index=False)

In [46]:
public_id = list(df['sig_id'].values)
test_id = list(test_features['sig_id'].values)
private_id = list(set(test_id)-set(public_id))
df_submit = pd.DataFrame(index = public_id+private_id, columns=target_cols)
df_submit.index.name = 'sig_id'
df_submit[:] = 0
df_submit.loc[test.sig_id,:] = test[target_cols].values
df_submit.loc[test_features[test_features.cp_type=='ctl_vehicle'].sig_id]= 0
df_submit.to_csv('submission_model5.csv',index=True)

# Your support motivates me to share kernels like these ... so please " Do UPVOTE "